In [ ]:
import collections

In [2]:
symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']

In [3]:
raw_token_freqs = {'fast_': 4, 'faster_':3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_token_freqs.items():
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

In [19]:
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int)
    for token, freq in token_freqs.items():
        symbols = token.split()
        for i in range(len(symbols) - 1):
            # "pairs"的键是两个连续符号的元组
            pairs[symbols[i], symbols[i+1]] += freq
    return max(pairs, key=pairs.get)  # 具有最大值的“pairs”键

In [20]:
# 作为基于连续符号频率的贪心方法，字节对编码将使用以下merge_symbols函数来合并最频繁的连续符号对以产生新符号
def merge_symbols(max_freq_pair, token_freqs, symbols):
    symbols.append(''.join(max_freq_pair))
    new_token_freqs = dict()
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                   ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

In [21]:
# 现在，我们对词典token_freqs的键迭代地执行字节对编码算法。在第一次迭代中，最频繁的连续符号对是‘t' 和’a'，因此字节对编码将它们合并以产生新符号‘ta'。在第二次迭代中，字节对编码继续合并’ta'和‘l’易产生另一个新符号‘tal’
num_merges = 10
for i in range(num_merges):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols)
    print(f'合并# {i+1}:', max_freq_pair)

合并# 1: ('t', 'a')
合并# 2: ('ta', 'l')
合并# 3: ('tal', 'l')
合并# 4: ('f', 'a')
合并# 5: ('fa', 's')
合并# 6: ('fas', 't')
合并# 7: ('e', 'r')
合并# 8: ('er', '_')
合并# 9: ('tall', '_')
合并# 10: ('fast', '_')


In [22]:
# 在字节对编码的十次迭代之后，我们可以看到列表symbols现在又包含10个从其他符号迭代合并而来的符号
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


In [23]:
# 对于在词典raw_token_freqs的键中制定的同一数据集，作为字节对编码算法的结果，数据集中的每个词现在被字词“fast_”"fast""er_""tall_""tall"分割
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']


In [24]:
token_freqs

{'fast_': 4, 'fast er_': 3, 'tall_': 5, 'tall er_': 4}

In [42]:
# 字节对编码的结果取决于正在使用的数据集。我们还可以使用从一个数据集学习的子词来切分另一个数据集的单词。作为一种贪心方法，下面的segment_BPE函数尝试将单词从输入参数symbols分成可能最长的子词。
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end=0, len(token)
        cur_output = []
        # 具有符号中可能最长子字的词元段
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                print(token[start:end])
                cur_output.append(token[start:end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        print(cur_output)
        outputs.append(' '.join(cur_output))
    return outputs

In [46]:
tokens = ['tallest_', 'fatter_', 'hello']
segment_BPE(tokens, symbols)

tall
e
s
t
_
['tall', 'e', 's', 't', '_']
fa
t
t
er_
['fa', 't', 't', 'er_']
h
e
l
l
o
['h', 'e', 'l', 'l', 'o']


['tall e s t _', 'fa t t er_', 'h e l l o']

In [29]:
symbols

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '_',
 '[UNK]',
 'ta',
 'tal',
 'tall',
 'fa',
 'fas',
 'fast',
 'er',
 'er_',
 'tall_',
 'fast_']